In [20]:
from neo4j import GraphDatabase

uri = "neo4j://10.195.2.227:7687"
username = "neo4j"
password = "neo4j-connect"

driver = GraphDatabase.driver(uri, auth=(username, password))

def add_node(session, node):
    result = session.run("CREATE (a:Wiki $node)", node=node)
        
def add_edge(session, docid1, docid2, edge):
    result = session.run("MATCH (a:Wiki {docid: $docid1}) MATCH (b:Wiki {docid: $docid2}) MERGE (a)-[:$edge]->(b)", docid1=docid1, docid2=docid2, edge=edge)

In [21]:
import json
from tqdm import tqdm
import time
import gc

s_time = time.time()
data = json.load(open("C:\\Users\\96181\\Desktop\\Master's Thesis\\T-Rex-Sample-Data.json", "r+"))
with driver.session() as session:
    for d in tqdm(data):
        entry = {
            'docid': d['docid'],
            'title': d['title'],
            'text': d['text'],
            'uri': d['uri']
        }
        add_node(session, entry)
        
print(f"Time for file {file} is {int(e_time - time.time())}sec")
del data
gc.collect()

  6%|████▍                                                                         | 567/10000 [00:43<12:00, 13.09it/s]


BufferError: Existing exports of data: object cannot be re-sized

In [15]:
data = json.load(open("C:\\Users\\96181\\Desktop\\Master's Thesis\\T-Rex-Sample-Data.json", "r+"))
data[0]['title']

'Austroasiatic languages'

In [16]:
data[0]['text']

'The Austroasiatic languages, in recent classifications synonymous with Mon–Khmer, are a large language family of continental Southeast Asia, also scattered throughout India, Bangladesh, Nepal and the southern border of China. The name Austroasiatic comes from the Latin words for "south" and "Asia", hence "South Asia". Of these languages, only Vietnamese, Khmer, and Mon have a long-established recorded history, and only Vietnamese and Khmer have official status (in Vietnam and Cambodia, respectively). The rest of the languages are spoken by minority groups. Ethnologue identifies 168 Austroasiatic languages. These form thirteen established families (plus perhaps Shompen, which is poorly attested, as a fourteenth), which have traditionally been grouped into two, as Mon–Khmer and Munda. However, one recent classification posits three groups (Munda, Nuclear Mon-Khmer and Khasi-Khmuic) while another has abandoned Mon–Khmer as a taxon altogether, making it synonymous with the larger family. 

In [2]:
from sentence_transformers import SentenceTransformer, util

query = "How many people live in London?"
docs = ["Around 9 Million people live in London", "London is known for its financial district"]

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)

27.682634353637695 Around 9 Million people live in London
16.25379753112793 London is known for its financial district


In [13]:
doc_emb.tolist()[0]

[-0.06503904610872269,
 -0.020100340247154236,
 -0.32590508460998535,
 -0.03615710884332657,
 0.13355626165866852,
 -0.054933302104473114,
 0.48720023036003113,
 -0.084445059299469,
 0.16023895144462585,
 0.24773827195167542,
 -0.032477088272571564,
 -0.09693075716495514,
 -0.30164745450019836,
 -0.10456578433513641,
 0.24363799393177032,
 -0.14142826199531555,
 0.23013168573379517,
 -0.0735398530960083,
 -0.5380126237869263,
 -0.055968139320611954,
 0.02962271124124527,
 -0.23075680434703827,
 -0.02918170765042305,
 -0.04759618639945984,
 -0.10320217162370682,
 0.2504409849643707,
 0.10331809520721436,
 0.005092129111289978,
 0.02194073237478733,
 0.4023004174232483,
 0.03755924850702286,
 -0.21172171831130981,
 -0.03931073099374771,
 0.4320336580276489,
 -0.00010358101280871779,
 0.23151235282421112,
 0.23523208498954773,
 0.009740548208355904,
 0.09262724220752716,
 0.07545197010040283,
 -0.24384388327598572,
 0.14552459120750427,
 -0.15467794239521027,
 0.23124960064888,
 0.0085013

In [63]:
from transformers import AutoTokenizer, AutoModel
import torch

#CLS Pooling - Take output from first token
def cls_pooling(model_output):
    return model_output.last_hidden_state[:,0]

#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = cls_pooling(model_output)

    return embeddings


# Sentences we want sentence embeddings for
query = "How many people live in London?"
docs = ["Around 9 Million people live in London", "London is known for its financial district"]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-mpnet-base-dot-v1")
model = AutoModel.from_pretrained("sentence-transformers/multi-qa-mpnet-base-dot-v1")

#Encode query and docs
query_emb = encode(query)
doc_emb = encode(docs)

#Compute dot score between query and all document embeddings
scores = torch.mm(query_emb, doc_emb.transpose(0, 1))[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)


Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

C:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\96181\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [50]:
data = json.load(open("C:\\Users\\96181\\Desktop\\Master's Thesis\\T-Rex-Sample-Data.json", "r+"))

In [56]:
def add_edge(session, docid1, docid2, edge):
    edge_name = edge['predicate_name'].replace(' ', '_')
    result = session.run(f"MATCH (a:Wiki {{docid: $docid1}}) MATCH (b:Wiki {{docid: $docid2}}) MERGE (a)-[r:{edge_name}]->(b) SET r += $edge", docid1=docid1, docid2=docid2, edge=edge)

predicate_names = {}
def get_predicate_name(uri):
    prop_id = uri.split('/')[-1]
    if prop_id in predicate_names:
        return predicate_names[prop_id]
    
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={prop_id}&format=json"
    response = requests.get(url)
    label = response.json()['entities'][prop_id]['labels']['en']['value']
    predicate_names[prop_id] = label
    return label

with driver.session() as session:
    t = data[0]['triples'][0]
    predicate_name = get_predicate_name(t['predicate']['uri'])
    entry = {
        'predicate_surfaceform': t['predicate']['surfaceform'],
        'predicate_uri': t['predicate']['uri'],
        'predicate_name': predicate_name,
        'object_surfaceform': t['object']['surfaceform'],
        'subject_surfaceform': t['subject']['surfaceform'],  
    }
    add_edge(session, t['subject']['uri'], t['object']['uri'], entry)

In [61]:
def get_predicate_name(uri):
    prop_id = uri.split('/')[-1]
    if prop_id in predicate_names:
        return predicate_names[prop_id]
    
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={prop_id}&format=json"
    response = requests.get(url)
    label = response.json()['entities'][prop_id]['labels']['en']['value']
    predicate_names[prop_id] = label
    return label

In [48]:
import requests

prop_id = 'P31'
url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={prop_id}&format=json"
response = requests.get(url)
label = response.json()['entities'][prop_id]['labels']['en']['value']
label

'instance of'

In [19]:
text = """text: Khas-Magomed Hadjimuradov (Chechen: Хас-Магомед Хаджимурадов) (born December 18, 1953 in Kazakhstan) is a Chechen nationalist bard. Khas-Magomed was born in the Zhambyl Province of Kazakhstan to Chechen parents, who had been forcibly removed from their homes in Chechnya as the result of the forced deportations of the Chechen and Ingush people to Central Asia on February 23, 1944. He spent much of his early youth in the village of Panfilovka in the Chuy Province of neighboring Kyrgyzstan, before his family returned their native village of Novye Atagi. After graduating from secondary school in 1971, he served in the Red Army from 1973 to 1975. In order to bolster the national identity of the Chechen people, Khas-Magomed performed nationalistic songs of Chechen heroes on Soviet television, hoping to revive the national consciousness of the Chechens and to restore "k'onalah", or the code of ethics in Chechen culture."""

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import sent_tokenize
from sklearn.cluster import AgglomerativeClustering

# Split the text into sentences
sentences = sent_tokenize(text)

# Vectorize the sentences using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)

# Use hierarchical clustering to chunk sentences
clustering = AgglomerativeClustering(n_clusters=2).fit(X.toarray())
labels = clustering.labels_

# Group sentences by cluster for final chunks
chunks = {}
for sentence, label in zip(sentences, labels):
    if label in chunks:
        chunks[label].append(sentence)
    else:
        chunks[label] = [sentence]

for label, chunk_sentences in chunks.items():
    print(' '.join(chunk_sentences))
    print()

text: Khas-Magomed Hadjimuradov (Chechen: Хас-Магомед Хаджимурадов) (born December 18, 1953 in Kazakhstan) is a Chechen nationalist bard.

Khas-Magomed was born in the Zhambyl Province of Kazakhstan to Chechen parents, who had been forcibly removed from their homes in Chechnya as the result of the forced deportations of the Chechen and Ingush people to Central Asia on February 23, 1944. He spent much of his early youth in the village of Panfilovka in the Chuy Province of neighboring Kyrgyzstan, before his family returned their native village of Novye Atagi. After graduating from secondary school in 1971, he served in the Red Army from 1973 to 1975. In order to bolster the national identity of the Chechen people, Khas-Magomed performed nationalistic songs of Chechen heroes on Soviet television, hoping to revive the national consciousness of the Chechens and to restore "k'onalah", or the code of ethics in Chechen culture.



In [49]:
from nltk.tokenize import sent_tokenize, word_tokenize
import re

def merged_sentence(words):
    if type(words[0]) == str:
        sentence = ' '.join(words)
        sentence = re.sub(r' ([''`"'']),', r'\1,', sentence)
        sentence = re.sub(r'\s([.,;:!?])', r'\1', sentence)
        sentence = re.sub(r'\s([\'])\s', r'\1', sentence)
        sentence = re.sub(r'\s([(])\s', r' \1', sentence)
        sentence = re.sub(r'\s([)])\s', r'\1 ', sentence)
        return sentence
    else:
        return ' '.join([merged_sentence(w) for w in words])

SENTENCE_SIZE = 124
OVERLAP_SIZE = 32
def chunk_text(text):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_chunk_size = 0
    for sentence in sentences:
        words = word_tokenize(sentence)
        # Estimate if adding the next sentence will exceed the token limit
        if current_chunk_size + len(words) > SENTENCE_SIZE:
            # Add current_chunk to chunks and start a new chunk
            chunks.append(merged_sentence(current_chunk))
            overlap = current_chunk[-1][-OVERLAP_SIZE:] if len(current_chunk[-1]) > OVERLAP_SIZE else current_chunk[-1]  # Get last 64 characters or full chunk
            current_chunk = [overlap, words] # Overlap
            current_chunk_size = len(overlap) + len(words)
        else:
            current_chunk.append(words)  # Add the next sentence to the current chunk
            current_chunk_size += len(words)
    chunks.append(merged_sentence(current_chunk))  # Add the last remaining chunk
    return chunks

chunk_text(text)

['text: Khas-Magomed Hadjimuradov (Chechen: Хас-Магомед Хаджимурадов) (born December 18, 1953 in Kazakhstan) is a Chechen nationalist bard. Khas-Magomed was born in the Zhambyl Province of Kazakhstan to Chechen parents, who had been forcibly removed from their homes in Chechnya as the result of the forced deportations of the Chechen and Ingush people to Central Asia on February 23, 1944. He spent much of his early youth in the village of Panfilovka in the Chuy Province of neighboring Kyrgyzstan, before his family returned their native village of Novye Atagi. After graduating from secondary school in 1971, he served in the Red Army from 1973 to 1975.',
 "After graduating from secondary school in 1971, he served in the Red Army from 1973 to 1975. In order to bolster the national identity of the Chechen people, Khas-Magomed performed nationalistic songs of Chechen heroes on Soviet television, hoping to revive the national consciousness of the Chechens and to restore `` k'onalah '', or the